# COMS 4995_002 Deep Learning Assignment 1
Due on Monday, Oct 9, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Wilson Lui, wl2522

Member 2: Vanessa Saldana Fountain, vsf2106

Member 3: Craig Brandon Barretto, cbb2151

In [50]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [96]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        self.parameters = {'num_layers': self.num_layers,
                           'drop_prob': self.drop_prob,
                           'reg_lambda': self.reg_lambda}
        
        self.weights = [np.random.randn(layer_dimensions[layer],
                                        layer_dimensions[layer - 1]) for layer in range(1, self.num_layers)]
        self.biases = [np.random.randn(layer, 1) for layer in layer_dimensions[1:]]
        
        # init parameters

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        Z = np.matmul(W, A) + b
        cache = [A, W, b]
        
        return Z, cache
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        if activation == 'relu':
            return self.relu(self.dropout(A, self.drop_prob)[0])
        
        elif activation == 'relu_no_dropout':
            return self.relu(A)

        
    def relu(self, X):
        return np.maximum(0, X)
            
        
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.binomial(1, 1 - prob, A.shape)
        A = (M*A)/(1 - prob)

        return A, M

    
    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        
        weights = self.weights
        biases = self.biases
        
        cache = []
        AL = X

        for i in range(1, self.num_layers):
            h, c = self.affineForward(AL, weights[i - 1], biases[i - 1])
            cache.append(c)
            
            if (i - 1) == 0 or i == (self.num_layers - 1):
                AL = self.activationForward(h, activation='relu_no_dropout')
            
            else:
                AL = self.activationForward(h)
            
        return AL, cache


    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        
        numerator = AL[y.astype(int), np.arange(y.shape[0])]
        denominator = scipy.misc.logsumexp(AL, axis=0)
        a = np.exp(numerator - denominator)
        
        if self.reg_lambda > 0:
            # add regularization
            l2 = 0
            
            for W in self.weights:
                l2 += np.sum(np.power(W, 2))
            
            l2 = l2 + self.reg_lambda/(2*y.shape[0])
            cost = np.sum(-np.log(a))/AL.shape[1] + l2
        
        elif self.reg_lambda == 0:
            cost = np.sum(-np.log(a))/AL.shape[1]
        
        # gradient of cost
        dAL = a - np.where(np.argmax(AL, axis=0) == y, 1, 0)
        #dAL = -(y/a) - (1 - y)/(1 - a)
        
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        #fully connected layer with no non-linear activation function 
        #dot product between inputs and weights
        
        A = cache[0]
        W = cache[1]
        b = cache[2]
        
        dA = dA_prev.dot(dW.T).reshape(A.shape[0])
        dW = A.reshape(A.shape[0], -1).T.dot(dA_prev)
        db = np.sum(dA_prev, axis = 0)
        
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        if activation == 'relu':
            return relu(dropout_backward(self, dA, self.cache)[1])
        
        else:
            return dropout_backward(self, dA, self.cache)[1]
        
    def relu_derivative(self, dx, cached_x):
        dx = np.where(cached_x > 0)
        
        return dx

    def dropout_backward(self, dA, cache):
        dA = A*self.cache
        
        return dA 

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {'dW': [np.zeros(w.shape) for w in self.weights],
                     'db': [np.zeros(b.shape) for b in self.biases]}
        w_len = (self.num_layers) - 1
        for i in range(w_len):
            if self.drop_prob > 0:
                return self.dropout_backward(dAL, cache)
                #call dropout_backward
            
        if self.reg_lambda > 0:
            dW += dAL*self.weights
            # add gradients from L2 regularization to each dW
            
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        
        self.weights = [w - (alpha*dw) for w, dw in zip(self.weights,gradients['dW'])]
        self.biases = [b - (alpha*db) for b, db in zip(self.biases,gradients['db'])]

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        num_samples = len(y)
        idx = np.random.randint(num_samples, size=batch_size)
        
        X_batch = X[:,idx]
        y_batch = y[idx]

        return X_batch, y_batch
    
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        tmp = np.vstack((X, y.reshape(1, y.shape[0])))
        np.random.shuffle(tmp.T)
        split_point = -int(0.1*X.shape[1])
        X_train = tmp[:-1, :split_point]
        y_train = tmp[-1, :split_point]
        X_val = tmp[:-1, split_point:]
        y_val = tmp[-1, split_point:]
        
        for i in range(0, iters):
            # get minibatch
            X_batch, y_batch = self.get_batch(X_train, y_train, batch_size)
            
            # forward prop
            AL, cache = self.forwardPropagation(X_batch)

            # compute loss
            cost, dAL = self.costFunction(AL, y_batch)

            # compute gradients
            gradients = self.backPropagation(dAL, y_batch, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)

            if i % print_every == 0:
                train_pred = self.predict(X_train)
                train_acc = np.where(train_pred == y_train)[0].shape[0]/y_train.shape[0]
                val_pred = self.predict(X_val)
                val_acc = np.where(val_pred == y_val)[0].shape[0]/y_val.shape[0]
                
                # print cost, train and validation set accuracies
                print('cost: {}, training accuracy: {}, validation accuracy: {}'.format(cost, train_acc, val_acc))
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        #y_pred = [np.argmax(self.forwardPropagation(x)) for x in X]
        y_pred = list()
        for x in range(X.shape[1]):
            pred = np.argmax(self.forwardPropagation(X[:, x].reshape(-1, 1))[0])
            y_pred.append(pred)
        
        return y_pred


In [3]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [4]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [15]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'frog': 6, 'truck': 9, 'horse': 7, 'bird': 2, 'ship': 8, 'dog': 5, 'airplane': 0, 'automobile': 1, 'cat': 3, 'deer': 4}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [97]:
layer_dimensions = [X_train.shape[0], 200, 100, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=1, alpha=0.00001, batch_size=100, print_every=5)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:131: RuntimeWarning: divide by zero encountered in log


cost: inf, training accuracy: 0.09713333333333334, validation accuracy: 0.101


In [21]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

KeyboardInterrupt: 

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [92]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0.5, reg_lambda=0)
NN2.train(X_train[:, :500], y_train[:500], iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

[  0.00000000e+000   2.23116945e-257   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   1.14465097e-040   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   4.70995572e-092   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   4.19516544e-309   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000
   3.23723084e-304   0.00000000e+000   0.00000000e+

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:131: RuntimeWarning: divide by zero encountered in log


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')